In [1]:
# Week 6 Optional Part 4 - subpart 2
# Raster to Numpy Array Example 2 from https://desktop.arcgis.com/en/arcmap/latest/analyze/arcpy-functions/rastertonumpyarray-function.htm
# 
# Input .tif file boston_01_Clip1.tif has 3 bands
# Output .tif can be successfully viewed in ArcGIS Pro

In [7]:
# Note that, if the input raster is multiband, the data blocks will also be
# multiband, having dimensions (bands, rows, columns).  Otherwise, they will
# have dimensions of (rows, columns).

import arcpy
import numpy
import os

# Input raster
filein = os.path.join(os.getcwd(),r"C:\Users\dshih\Documents\a_GIS6345\Week6\boston\boston_01_Clip1.tif")


# Output raster (after processing)
fileout = os.path.join(os.getcwd(),r"C:\Users\dshih\Documents\a_GIS6345\Week6\boston\output_block3.tif")

# Size of processing data block
#  where memorysize = datatypeinbytes * numbands * blocksize^2
blocksize = 512

# ----------------------------------------------------------------------------
# Create raster object from file
myRaster = arcpy.Raster(filein)  #type(myRaster) is an arcpy.sa.Raster.Raster

# Set environmental variables for output
arcpy.env.overwriteOutput = True #type(arcpy.env.overwriteOutput) is bool
arcpy.env.outputCoordinateSystem = filein #type(arcpy.env.outputCoordinateSystem)
# value is arcpy.arcobjects.arcobjects.SpatialReference # Projected WGS_1984_UTM_Zone_19N
arcpy.env.cellSize = filein  # type(arcpy.env.cellSize) is str
# value is 'C:\\Users\\dshih\\Documents\\a_GIS6345\\Week6\\boston\\boston_01_Clip1.tif'



In [8]:
# Loop over data blocks
filelist = []
blocknum = 0
for x in range(0, myRaster.width, blocksize):
    for y in range(0, myRaster.height, blocksize):

        # Lower left coordinate of block (in map units)
        mx = myRaster.extent.XMin + x * myRaster.meanCellWidth #type(mx) is float # value is 301065.0
        my = myRaster.extent.YMin + y * myRaster.meanCellHeight #type(my) is float # value is 4665675.0
        # type(myRaster.extent) is arcpy.arcobjects.arcobjects.Extent
        # type(myRaster.extent.XMin) is float # value is <Extent object at (address[address])>
        # value of myRaster.extent.XMin is 301065.0, value of myRaster.extent.YMin is 4665675.0
        # Upper right coordinate of block (in cells)
        lx = min([x + blocksize, myRaster.width])  #type is int #value is 512
        ly = min([y + blocksize, myRaster.height]) #type is int #value is 512
        # myRaster.width is 1660, myRaster.height is 1632, both are type int
        #noting that (x, y) is the lower left coordinate (in cells)

        # Extract data block
        myData = arcpy.RasterToNumPyArray(myRaster, arcpy.Point(mx, my),lx-x, ly-y) #type is numpy.ndarray
        # myData.shape is (3, 512, 512)
        # type(myData[0]) and [0,0] is numpy.ndarray 
        # type(myData[0,0,0]) is numpy.int32
        # type(arcpy.Point(mx, my) is arcpy.arcobjects.arcobjects.Point
        # value of arcpy.Point(mx,my) is <Point (301065.0, 4665675.0, #, #)


        #the next code caused an UFuncTypeError: Cannot cast 
        # ufunc 'subtract' output from dtype('float64') to 
        # dtype('int32') with casting rule 'same_kind'
        # From a techoverflow.net posting, replace the -= 
        
        # PROCESS DATA BLOCK -----------------------------
        # e.g., Calculate mean of each cell of all bands.
        # This causes error
        # myData -= numpy.mean(myData, axis=0, keepdims=True)
        # type(numpy.mean(myData, axis=0, keepdims=True)) is numpy.ndarray
        # type(numpy.mean(myData, axis=0, keepdims=True)[0] and also [0,0] are numpy.ndarray
        # type(numpy.mean(myData, axis=0, keepdims=True)[0,0,0]) is numpy.float64
        # changed to:
        myData = myData - numpy.mean(myData, axis=0, keepdims=True)
        # ------------------------------------------------

        # Convert data block back to raster
        myRasterBlock = arcpy.NumPyArrayToRaster(myData, arcpy.Point(mx, my),
                                                 myRaster.meanCellWidth,
                                                 myRaster.meanCellHeight)

        # type(myRasterBlock) is arcpy.sa.Raster.Raster
        
        # Save on disk temporarily as 'filename_#.ext'
        filetemp = ('_%i.' % blocknum).join(fileout.rsplit('.',1))
        myRasterBlock.save(filetemp)

        # Maintain a list of saved temporary files
        filelist.append(filetemp)
        blocknum += 1

# Mosaic temporary files
arcpy.Mosaic_management(';'.join(filelist[1:]), filelist[0])
if arcpy.Exists(fileout):
    arcpy.Delete_management(fileout)
arcpy.Rename_management(filelist[0], fileout)

# Remove temporary files
for fileitem in filelist:
    if arcpy.Exists(fileitem):
        arcpy.Delete_management(fileitem)

# Release raster objects from memory
del myRasterBlock
del myRaster
# ----------------------------------------------------------------------------